Compare no power control with UD-DPC in the noise-limited and interferencelimited scenarios. Set $P_{max}$ = 30 dBm, $P_{min}$ = 0 dBm and the discrete power control
update step as $\mu$ = 1 dBm. As for the target SINR required in UD-DPC, let us assume
the value corresponding to the QoS target of 100 Mbps at cell border under 100 MHz
which equates to 
$\gamma_t$ = 1. All other simulation parameters are kept in the same values
used in previous exercises. What are your observations? Did fixed-target SINR-based
power control (UD-DPC) help in any of the scenarios?


In [13]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(12)

In [12]:
# Função que transforma Linear para dB
def lin2db(x):
    return 10 * np.log10(x)

# Função que transforma dB para Linear
def db2lin(x):
    return 10 ** (x / 10)

# Função que transforma Linear para dBm (watt)
def lin2dbm(x):
    return 10 * np.log10(x * 1000)

# Função que transforma dBm para Linear (watt)
def dbm2lin(x):
    return 10**(x / 10) / 1000

In [3]:
# Função que define os eixos x e y da CDF
def cdf(dados):
    '''Função que calcula a Cumulative Distribution Function (CDF) de um conjunto de dados.
    Parâmetros:
    dados: array. Conjunto de dados.
    Retorno:
    x: array. Dados ordenados.
    y: array. Probabilidade acumulada de cada dado.'''

    x = np.sort(dados)
    y = np.arange(1, len(x) + 1) / len(x)
    return x, y

In [4]:
# Função que gera as coordenadas dos APs
def distribuir_APs(num_aps, area):
    '''Distributes Access Points (APs) evenly within a square area.
    
    Parameters:
    num_aps (int): The number of APs to distribute. Must be a perfect square.
    area (int): The length of the side of the square area in which to distribute the APs.
    
    Returns:
    np.array: An array of coordinates for the APs, or None if num_aps is not a perfect square.'''
    
    if num_aps not in [1, 4, 9, 16, 25, 36, 49, 64, 100]:
        return None

    tamanho_quadrado = area
    lado_quadrado = int(np.sqrt(num_aps))

    tamanho_celula = tamanho_quadrado // lado_quadrado

    # Criar coordenadas usando meshgrid
    x, y = np.meshgrid(np.arange(0.5 * tamanho_celula, tamanho_quadrado, tamanho_celula),
                      np.arange(0.5 * tamanho_celula, tamanho_quadrado, tamanho_celula))

    coordenadas_APs = np.column_stack((x.ravel(), y.ravel()))

    return coordenadas_APs

In [5]:
# Função que gera a distância entre a UE e a AP
def dAPUE(x_coord, y_coord, ap_coord, d_reference=1):
  '''Calculate the Euclidean distance between a user equipment (UE) and an access point (AP).
    
    Parameters:
    ue_coords (tuple): A tuple (x_coord, y_coord) representing the coordinates of the UE.
    ap_coords (np.array): An array containing the coordinates of the APs.
    
    Returns:
    float: The Euclidean distance between the UE and the AP. If the euclidean distance is less than 1, return 1.
  '''
  dAPUE = np.linalg.norm(np.array([x_coord, y_coord]) - ap_coord)
  if dAPUE < d_reference:
    return d_reference
  else:
    return dAPUE

In [6]:
#Função que define o shadowing para cada usuário
def find_shadowing(standard_shadow, ues, aps):
    '''Generate a shadowing value for a user.

    Parameters:
    standard_shadow (float): The standard deviation of the shadowing values.
    ues (int): The number of user equipment (UE) in the simulation.
    aps (int): The number of access points (APs) in the

    Returns:
    np.array: A 2D array of shadowing values for each UE and AP.
    '''

    shadowing = np.random.lognormal(0, standard_shadow, (ues, aps))

    return shadowing 

In [7]:
def find_fastfading(standard_fading, ues, aps, channels):
    '''Função que calcula o fast fading para um dado usuário.
    
    Parâmetros:
    standard_fading (float): O desvio padrão da distribuição normal.
    ues (int): O número de usuários.
    aps (int): O número de pontos de acesso.
    channels (int): O número de canais.
    
    Retorna:
    np.array: Um array 3D com os valores de fast fading para cada usuário, ponto de acesso e canal.
    '''

    fast_fading = np.sqrt( (standard_fading * np.random.randn(channels, ues, aps))**2 + (standard_fading * np.random.randn(channels, ues, aps))**2 )

    return fast_fading

In [8]:
# Função que calcula o path gain com a distância, shadowing e fast fading
def find_pathgain(dist, shadowing, fastfading):
    ambiente_const = 1e-4
    pathloss_const = 4


    path_gain_result = (shadowing * (ambiente_const / (dist ** pathloss_const))) * fastfading **2

    
    return path_gain_result

In [9]:
def calculate_sinr(banda, K_0, aps, ues, channels, sigma_shadowing, sigma_fastfading, p_t, area):
    '''Função que calcula a SINR para múltiplos UEs e APs em diferentes canais.

    i-th AP, j-th UE, c-th channel
    
    Lembrando que [channels, ues, aps] é a dimensão de fastfading, logo as dimensões da matriz path_gain é [channels, ues, aps]. 

    Parâmetros:
    banda (float): Largura de banda total.
    K_0 (float): Constante de ruído.
    aps (int): Número de APs.
    ues (int): Número de UEs.
    channels (int): Número de canais.
    sigma_shadowing (float): Desvio padrão para o shadowing.
    sigma_fastfading (float): Desvio padrão para o fast fading.
    p_t (float): Potência de transmissão.
    rrm_index (int): Índice para o tipo de algoritmo de alocação de recursos.
    area (float): Área de distribuição dos APs.
    
    Retorna:
    np.array: O valor do SINR para cada UE.
    '''
    # Inicializações
    x_coord, y_coord, aps_position = np.zeros(ues), np.zeros(ues), distribuir_APs(aps, area)
    dist = np.zeros((ues, aps))
    shadowing = find_shadowing(sigma_shadowing, ues, aps)
    fastfading = find_fastfading(sigma_fastfading, ues, aps, channels)
    power_trans = np.ones(ues) * p_t
    power_noise = np.ones(ues) * (K_0 * banda / channels)

    # Coordenadas dos UEs
    for ue_index in range(ues):
        x_coord[ue_index] = np.random.randint(0, area)
        y_coord[ue_index] = np.random.randint(0, area)

    # Distância entre UEs e APs
    for ue_index in range(ues):
        for ap_index in range(aps):
            dist[ue_index][ap_index] = dAPUE(x_coord[ue_index], y_coord[ue_index], aps_position[ap_index])

    path_gain = find_pathgain(dist, shadowing, fastfading)

    # Cálculo do SINR
    sinr_total = np.zeros(path_gain.shape)
    sinr_ue = np.zeros(path_gain.shape[1])

    for ue_index in range(path_gain.shape[1]):
        for channel_index in range(path_gain.shape[0]):
            for ap_index in range(path_gain.shape[2]):
                # Calculando a potência recebida no canal atual
                power_received = np.abs(path_gain[channel_index, ue_index, ap_index]) * power_trans[ue_index]
    
                # Calculando a interferência no mesmo canal
                interference_sum = 0
                for other_ue_index in range(path_gain.shape[1]):
                    if other_ue_index != ue_index:  # Se não for o mesmo usuário
                        interference_sum += np.abs(path_gain[channel_index, other_ue_index, ap_index]) * power_trans[other_ue_index]

                # Cálculo da SINR para o canal, UE e AP atuais
                sinr_total[channel_index, ue_index, ap_index] = power_received / (interference_sum + power_noise[ue_index])

    # Selecionar o maior valor de SINR para cada UE
    for ue_index in range(sinr_total.shape[1]):
        sinr_ue[ue_index] = np.max(sinr_total[:, ue_index, :])

    return sinr_ue


In [ ]:
def updown_dpc(path_gain, aps, ues, channels, power_trans_max, power_trans_min, step, sinr_alvo, time, power_noise, banda, K_0, area, sigma_shadowing, sigma_fastfading):
    transmit_power = power_trans_max * np.ones(path_gain.shape[1], 1)
    power_alocation = np.zeros((ues, time))
    sinr_alocation = np.zeros((ues, time))

    for time_index in range(time):
        # Calcula o SINR para cada usuário para cada momento
        sinr_ue = calculate_sinr(banda, K_0, aps, ues, channels, sigma_shadowing, sigma_fastfading, dbm2lin(transmit_power), area)
        power_alocation[:, time_index] = transmit_power.flatten()

        transmit_power = transmit_power + step * np.sign((sinr_ue - sinr_alvo))
        
        if transmit_power > power_trans_max:
            transmit_power = power_trans_max
        elif transmit_power < power_trans_min:
            transmit_power = power_trans_min

    return transmit_power



In [10]:
def find_capacity(sinr, banda, channels):
    '''Função que calcula a capacidade de um dado canal.
    
    Parâmetros:
    sinr (list): A relação sinal ruído mais interferência.
    banda (int): A largura de banda do canal.    
    channels (int): O número de canais.
    Retorna:
    list: A capacidade do canal.'''

    banda_channel = banda / channels
    
    capacity = banda_channel * np.log2(1 + sinr)
    
    return capacity

In [11]:
bandwidth , transmit_power, d_min, K_0 = 100e6, 1, 1, 1e-20 # Em MHz, mW, metros, mW/Hz respectivamente
aps, ues, channels = 4, 4, 1
area = 1000
sigma_shadowing = 2
sigma_fastfading = 1/np.sqrt(2)
iterat = 1000

In [ ]:
power_trans_max = 30 # Em dBm
power_trans_min = 0 # Em dBm
step = 1 # Em dBm